<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/GAN/3DGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D - GAN
Conv2D 대신 Conv3D 를 사용하여 3D 데이터셋을 처리한 모델이다. 

일반 DCGAN 대신 LSGAN 을 사용하여 손실을 최소화 시키기 위해 loss 를 mse 를 사용하였다. 

추가로 최적화를 위해서는 아래의 논문을 보고 모델을 구성하길 바란다.

http://3dgan.csail.mit.edu/papers/3dgan_nips.pdf

데이터 셋은 해당 저자의 홈페이지에서 가입을 통해 다운로드 받을 수 있다.

In [ ]:
import glob
import os

import numpy as np
import scipy.io as io
import scipy.ndimage as nd
from tensorflow.keras.layers import Input, Activation, Flatten, Dense
from tensorflow.keras.layers import Conv3D, Conv3DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop

from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt

# GPU 할당

In [ ]:
import tensorflow as tf 
physical_devices =tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# 모델 구성함수

In [ ]:
def build_generator():
    z_size = 200
    layer_filters = [256, 128, 64]
    kernel_sizes = 4
    strides = 2
    input_shape = (1, 1, 1, z_size)
    
    inputs = Input(shape = input_shape, name = 'generator_input')
     
    # input 다음의 Conv3DTranspose BN ReLU 층
    x = Conv3DTranspose(filters = 512,
                       kernel_size = kernel_sizes,
                       strides = 1)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    # Conv3DTranspose 층 
    for filters in layer_filters:
        x = Conv3DTranspose(filters=filters,
                           kernel_size = kernel_sizes,
                           strides = strides,
                           padding='same')(x)
        
        x = BatchNormalization()(x)
        x = Activation(activation = 'relu')(x)
    
    # 마지막 층은 활성화 함수를 SIgmoid 로 한다.
    x = Conv3DTranspose(filters = 1, 
                          kernel_size = kernel_sizes, 
                          strides = strides,
                          padding = 'same')(x)

    x = Activation(activation='sigmoid')(x)
    
    return Model(inputs, x, name='generator')

def build_discriminator():
    input_shape = (64,64,64,1)
    layer_filters = [64, 128, 256]
    kernel_sizes = 4
    strides = 2
        
    #LeakyReLU 의 alpha 값
    alphas = 0.2
    
    inputs = Input(shape=input_shape, name= "discriminator_input")
    
    # 첫번째 Conv3D 층
    x = Conv3D(filters = 32, 
               kernel_size=kernel_sizes,
               strides=strides,
               padding='same')(inputs)
    
    #x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=alphas)(x)
    
    for filters in layer_filters:
        x = Conv3D(filters = filters,
                   kernel_size = kernel_sizes,
                   strides = strides,
                   padding = 'same')(x)
        x = LeakyReLU(alpha=alphas)(x)
    
    x = Conv3D(filters=512, kernel_size = kernel_sizes, strides = strides, padding = 'same')(x)
    
    return Model(inputs, x, name = 'discriminator')

# 유틸함수 정의

In [ ]:
def get3DImages(data_dir):
    all_files = np.random.choice(glob.glob(data_dir), size=500)
    #all_files = glob.glob(data_dir)
    all_volumes = np.asarray([getVoxelsFromMat(f) for f in all_files], dtype = np.bool)
    
    return all_volumes

def getVoxelsFromMat(path, cube_len=64):
    voxels = io.loadmat(path)['instance']
    voxels = np.pad(voxels, (1,1), 'constant', constant_values = (0,0))
    if cube_len != 32 and cube_len == 64:
        voxels = nd.zoom(voxels, (2,2,2),mode='constant', order=0)
    
    return voxels

def saveFromVoxels(voxels,path):
    x,y,z = voxels.nonzero()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, -z, zdir='z', c='green')
    
    plt.savefig(path)
    
def plotAndSaveVoxel(file_path, voxel):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')
    ax.voxels(voxel, edgecolor="green")
    #plt.show()
    plt.savefig(file_path)

# 데이터 디렉토리 및 모델 파라미터 지정

In [ ]:
object_name = "airplane"
data_dir = "D:/data_sets/3DShapeNets/volumetric_data/{}/30/train/*.mat".format(object_name)
print(data_dir)

lr = 2e-4
decay = 6e-8
beta = 0.5
batch_size = 10
z_size = 200
epochs = 500

# 모델 구성

In [ ]:
#모델 옵티마이저 정의
optimizer = RMSprop(lr=lr, decay=decay)

#판별기 구성
discriminator = build_discriminator()
discriminator.compile(loss='mse',optimizer=optimizer, metrics=['accuracy'])
discriminator.summary()

In [ ]:
#생성기 구성
generator = build_generator()
generator.summary()

In [ ]:
#적대적 모델 구성
optimizer = RMSprop(lr=0.5*lr, decay=0.5*decay)

discriminator.trainable = False
inputs = Input(shape = (1, 1, 1, z_size), name='z_input')
adversarial = Model(inputs, discriminator(generator(inputs)), name='3D-GAN')

adversarial.compile(loss = 'mse', optimizer=optimizer,metrics=['accuracy'])
adversarial.summary()

# 데이터 로딩

In [ ]:
# 데이터 로딩
volumes = get3DImages(data_dir=data_dir)
volumes = volumes[...,np.newaxis].astype(np.float)

train_size = volumes.shape[0]
print(train_size)

# 모델 훈련

In [ ]:
models = (generator, discriminator, adversarial)
params = (batch_size, z_size ,epochs)

In [ ]:
# 모델 훈련 
def train(models, x_train, params):
    generator, discriminator, adversarial = models
    batch_size, z_size, epochs = params    
    
    train_size = x_train.shape[0]
    number_of_batches = int(x_train.shape[0]/batch_size)
    
    real_labels = np.ones([batch_size,1])
    fake_labels = np.zeros([batch_size,1])
    
    for epoch in range(epochs):     
              
        print("epoch : ", (epoch+1))
        
        for index in range (number_of_batches):
            process = round((index+1)/number_of_batches, 3)
            rep_star = "["+int(process*20)*"="+int((1-process)*20)*" "+"]"+str(process)
            volumes_batch = x_train[index*batch_size:(index+1)*batch_size,:,:,:]
            
            z_sample = np.random.uniform(-1.0, 1.0, size=[batch_size,1,1,1,z_size])
            
            gen_volumes = generator.predict_on_batch(z_sample)
            discriminator.trainable = True
            real_loss, real_acc = discriminator.train_on_batch(volumes_batch, real_labels)   
            fake_loss, fake_acc = discriminator.train_on_batch(gen_volumes, fake_labels)
            
            d_loss = 0.5*(real_loss+fake_loss)
            d_acc = 0.5*(real_acc+fake_acc)
            
            discriminator.trainable = False
            z = np.random.uniform(-1.0, 1.0, size=[batch_size,1,1,1,z_size])
            g_loss, g_acc = adversarial.train_on_batch(z, real_labels)
            
            log = "[discriminator loss : %f, acc: %f] [adversarial loss: %f, acc: %f] process:%s" %(d_loss, d_acc, g_loss, g_acc, rep_star)
            print(log, end="\r")
            
        print(log)
        
        if (epoch+1)%10 ==0:
            z_sample2 = np.random.normal(0, 0.33, size = [batch_size, 1, 1, 1, z_size]).astype(np.float32)
            generated_volumes = generator.predict(z_sample2, verbose=3)
            
            for i, generated_volume in enumerate(generated_volumes[:5]):
                voxels = np.squeeze(generated_volume)
                voxels[voxels<0.5] = 0.
                voxels[voxels>=0.5] = 1.
                saveFromVoxels(voxels, "D:/data_sets/3D_results/img_{}_{}".format(epoch+1,i))
                
    generator.save('3D_generator.h5')

In [ ]:
train(models, volumes, params)